In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import functional as F
import torch.optim as optim
import tensorboard

In [23]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(
            combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width,
                            device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width,
                            device=self.conv.weight.device))


class EncoderDecoderConvLSTM(nn.Module):

    def __init__(self, nf, in_chan, out_chan):
        super(EncoderDecoderConvLSTM, self).__init__()

        self.encoder_1_convlstm = ConvLSTMCell(input_dim=in_chan,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.encoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_1_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_CNN = nn.Conv3d(in_channels=nf,
                                     out_channels=out_chan,
                                     kernel_size=(1, 3, 3),
                                     padding=(0, 1, 1))
        
    def autoencoder(self, x, past_steps, future_steps, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4):
        outputs = []

        for t in range(past_steps):
            h_t, c_t = self.encoder_1_convlstm(input_tensor=x[:, t, :, :],
                                              cur_state=[h_t,c_t])
            h_t2, c_t2 = self.encoder_2_convlstm(input_tensor=h_t,
                                                cur_state=[h_t2, c_t2])

        encoder_vector = h_t2

        for t in range(future_steps):
            h_t3, c_t3 = self.decoder_1_convlstm(input_tensor=encoder_vector,
                                                cur_state=[h_t3, c_t3])
            h_t4, c_t4 = self.decoder_2_convlstm(input_tensor=h_t3,
                                                cur_state=[h_t4, c_t4])
            encoder_vector = h_t4
            outputs.append(h_t4)

        outputs = torch.stack(outputs, 1)
        outputs = outputs.permute(0, 2, 1, 3, 4)
        outputs = self.decoder_CNN(outputs)
        outputs = torch.nn.Sigmoid()(outputs)
        outputs = outputs.permute(0, 2, 1, 3, 4)
        
        return outputs
    
    def forward(self, x, future_steps=30, hidden_state=None):
        b, past_steps, _, h, w = x.size()
        
        h_t, c_t = self.encoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t2, c_t2 = self.encoder_2_convlstm.init_hidden(batch_size=b, image_size=(h,w))
        h_t3, c_t3 = self.decoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t4, c_t4 = self.decoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        
        outputs = self.autoencoder(x, past_steps, future_steps, h_t, c_t, h_t2, c_t2,
                                  h_t3, c_t3, h_t4, c_t4)

        return outputs

In [4]:
dataset = np.load('./splits/dataset.npy')

In [5]:
dataset_normalized = dataset.copy()

In [6]:
for i in range(8):
    dataset_normalized[:,:,i,:,:] = \
    (dataset_normalized[:,:,i,:,:] - np.nanmean(dataset_normalized[:,:,i,:,:])) / \
    np.nanstd(dataset_normalized[:,:,i,:,:])

In [7]:
dataset_normalized = np.nan_to_num(dataset_normalized, nan=0.)

In [8]:
dataset_normalized = dataset_normalized.astype('float32') 

In [9]:
np.save('./splits/dataset_normalized.npy', dataset_normalized)

In [11]:
X = torch.tensor(dataset_normalized[:,:11,:,:,:])
y = torch.tensor(dataset_normalized[:,11:,:,:,:])

In [12]:
ds = TensorDataset(X,y)

In [13]:
dl = DataLoader(ds, batch_size=64, shuffle=False)

In [14]:
len_train = int(len(ds) * 0.6)
len_valid = (len(ds) - len_train) // 2
len_test = (len(ds) - len_train) // 2

In [15]:
train_ds = torch.utils.data.Subset(ds, range(len_train))
valid_ds = torch.utils.data.Subset(ds, range(len_train,len_train+len_valid))
test_ds = torch.utils.data.Subset(ds, range(len_train+len_valid, len(ds)))

In [16]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=True)

In [21]:
n_epochs=100

In [18]:
writer = SummaryWriter()

In [19]:
%load_ext tensorboard

In [20]:
%tensorboard --logdir=runs

Reusing TensorBoard on port 6006 (pid 44155), started 0:26:22 ago. (Use '!kill 44155' to kill it.)

In [33]:
model = EncoderDecoderConvLSTM(nf=12, in_chan=8, out_chan=8)
model.to(device='cuda')

optimizer = optim.Adam(model.parameters())

loss_fn = nn.MSELoss()

def oxygen_mse(y_pred, y_true):
    return loss_fn(y_pred[:,:,1,:,:], y_true[:,:,1,:,:])

for epoch in range(n_epochs):
    for xs, ys in train_dl:
        xs.to(device='cuda')
        out = model(xs)
        train_loss = loss_fn(out, ys)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        writer.add_scalar('Loss/train', train_loss, epoch)

        
    if epoch % 50 == 0:
        torch.save(model.state_dict(), f'./saved_models/model_epoch_{epoch}.pt')
        
        with torch.no_grad():
            xs, ys = next(iter(valid_dl))
            xs.to(device='cuda')
            out = model(xs)
            val_loss = loss_fn(out, ys)
            val_o2_loss = oxygen_mse(out, ys)
            
            writer.add_scalar('Loss/validation', val_loss, epoch)
            writer.add_scalar('Loss/oxygen', val_o2_loss, epoch)

        print("Epoch: %d, Training loss: %f, Validation loss: %f, Validation MSE on O2: %f" %
              (epoch, float(train_loss), float(val_loss), float(val_o2_loss)))

AssertionError: Torch not compiled with CUDA enabled